In [23]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import pandas as pd

In [25]:
df = pd.read_csv('/Users/ryanqnelson/Documents/Holocron/_2020-CMU/courses/2021-fall/11-685 Deep Learning/homework/homework 2/submissions/submission02/Resnet50-Run-03.epoch46.20211013.14.31.47.output.csv')
imf = ImageFolder('/Users/ryanqnelson/Desktop/test/content/competitions/idl-fall21-hw2p2s1-face-classification/train_data')

In [38]:
mapping = imf.class_to_idx
# print(mapping)
# print(mapping.values())
# save this as separate json file / dictionary
list(mapping.keys())[list(mapping.values()).index(2)]

'10'

In [34]:
df['true_label']=df.apply(lambda x: list(mapping.keys())[list(mapping.values()).index(x['label'])], axis=1)
df

,id,label,true_label
0,0.jpg,3330,3996
1,1.jpg,1262,2133
2,2.jpg,3439,494
3,3.jpg,3113,380
4,4.jpg,1511,2358
...,...,...,...
7995,7995.jpg,1642,2476
7996,7996.jpg,32,1026
7997,7997.jpg,169,115
7998,7998.jpg,3722,749


In [35]:
df.to_csv('Resnet50-Run-03.epoch46.20211013.14.31.47.output.remapped.csv',header=True,index=False)